In [1]:
import cv2
import numpy as np
import time

#start reading webcam
cap = cv2.VideoCapture(0)

#read initial frames
ret, frame1 = cap.read()
ret, frame2 = cap.read()

#get screen dimensions
frame_height, frame_width = frame1.shape[:2]

#define center tolerance (I will need to test and tweak this)
center_tolerance = 50

while cap.isOpened():
    #compute the absolute difference between frames
    diff = cv2.absdiff(frame1, frame2)
    
    #convert the difference to grayscale
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    
    #apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    
    #get threshold
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    
    #dilate our threshold
    dilated = cv2.dilate(thresh, None, iterations=3)
    
    #get contours
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    #draw contours to detect motion
    for contour in contours:
        if cv2.contourArea(contour) < 700:
            continue
        
        #get bounding box coordinates for each contour
        (x, y, w, h) = cv2.boundingRect(contour)
        
        #mark object with a rectangle
        cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        #calculate center of bounding box
        center_x = x + w // 2
        center_y = y + h // 2
        
        #check if the object is (roughly) in the center of the screen
        screen_center_x = frame_width // 2
        screen_center_y = frame_height // 2
        
        if (abs(center_x - screen_center_x) <= center_tolerance and
            abs(center_y - screen_center_y) <= center_tolerance):
            #object is in the center of the screen, take a screenshot
            screenshot_filename = f'screenshot_{int(time.time())}.png'
            cv2.imwrite(screenshot_filename, frame1)
            print(f"Screenshot taken: {screenshot_filename}")
    
    #display resulting frame
    cv2.imshow("Motion Detection", frame1)
    
    #update frames
    frame1 = frame2
    ret, frame2 = cap.read()

    #exit loop
    if cv2.waitKey(10) == ord('q'):
        break

#close up program
cap.release()
cv2.destroyAllWindows()


Screenshot taken: screenshot_1726693691.png
Screenshot taken: screenshot_1726693691.png
Screenshot taken: screenshot_1726693691.png
Screenshot taken: screenshot_1726693693.png
Screenshot taken: screenshot_1726693693.png
Screenshot taken: screenshot_1726693693.png
Screenshot taken: screenshot_1726693693.png
Screenshot taken: screenshot_1726693695.png


KeyboardInterrupt: 

: 